# Mutable function parameters

- toc: true
- badges: true
- comments: true
- categories: [python]

*Notes based on chapter 8 in [Fluent Python](https://www.oreilly.com/library/view/fluent-python/9781491946237/).*

Functions that take mutable objects as arguments require caution, because function arguments are aliases for the passed arguments. This can cause unintended behaviour in two types of situations:

- When setting a mutable object as default
- When aliasing a mutable object passed to the constructor

## Setting a mutable object as default

In [5]:
class HauntedBus():
    
    def __init__(self, passengers=[]):
        self.passengers = passengers
        
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)
        

bus1 = HauntedBus(['pete', 'lara', 'nick'])
bus1.drop('nick')
print(bus1.passengers)

bus2 = HauntedBus()
bus2.pick('heather')
print(bus2.passengers)

bus3 = HauntedBus()
bus3.pick('katy')
print(bus3.passengers)

['pete', 'lara']
['heather']
['heather', 'katy']


Between bus 1 and 2, all works as intended, since we passed our own list when creating bus 1. Then things get a bit weird, though: how did Heather get into bus 3? When we define the `HauntedBus` class, we create a single empty list that is kept in the background and will be used whenever we instantiate a new bus without a custom passenger list. Importantly, all such buses will operate on the same list. We can see this by checking the object ids of the three buses' passenger lists:

In [17]:
assert bus1.passengers is not bus2.passengers
assert bus2.passengers is bus3.passengers

This shows that while the passenger list of bus 1 and 2 are not the same object, the lists of bus 2 and 3 are. Once we know that, the above behaviour makes sense: all passenger list operations on buses without a custom list operate on the same list. Anohter way of seeing this by inspecting the default dict of `HauntedBus` after our operations abve.

In [18]:
HauntedBus.__init__.__defaults__

(['heather', 'katy'],)

The above shows that after the `bus3.pick('katy')` call above, the default list is now changed, and will be inherited by future instances of `HauntedBus`.

In [19]:
bus4 = HauntedBus()
bus4.passengers

['heather', 'katy']

This behaviour is an example of why it matters whether we think of variables as boxes or labels. If we think that variables are boxes, then the above bevaviour doesn't make sense, since each passenger list would be its own box with its own content. But when we think of variables as labels -- the correct way to think about them in Python -- then the behaviour makes complete sense: each time we instantiate a bus without a custom passenger list, we create a new label -- of the form `name-of-bus.passengers` -- for the empty list we created when we loaded or created `HauntedBus`.

What to do to avoid the unwanted behaviour? The solution is to create a new empty list each time no list is provided.

In [23]:
class Bus():
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)
        
bus1 = Bus()
bus1.pick('tim')
bus2 = Bus()
bus2.passengers

[]

## Aliasing a mutable object argument inside the function

The `init` method of the above class copies the passed passenger list by calling `list(passengers)`. This is critical. If, instead of copying we alias the passed list, we change lists defined outside the function that are passed as arguments, which is probably not what we want.

In [24]:
class Bus():
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)
        
team = ['hugh', 'lisa', 'gerd', 'adam', 'emily']

bus = Bus(team)
bus.drop('hugh')
team

['lisa', 'gerd', 'adam', 'emily']

Again, the reason for this is that `self.passengers` is an alias for `passengers`, which is itself an alias for `team`, so that all operations we perfom on `self.passengers` are actually performed on `team`. The identity check below shows what the passengers attribute of `bus` is indeed the same object as the team list.

In [28]:
bus.passengers is team

True

**To summarise:** unless there is a good reason for an exception, for functions that take mutable objects as arguments do the following:

1. Create a new object each time a class is instantiated by using None as the default parameter, rather than creating the object at the time of the function definition.

2. Make a copy of the mutable object for processing inside the function to leave the original object unchanged.

## Main sources

- [Fluent Python](https://www.oreilly.com/library/view/fluent-python/9781491946237/)
- [Python Cookbook](https://www.oreilly.com/library/view/python-cookbook-3rd/9781449357337/)
- [Learning Python](https://www.oreilly.com/library/view/learning-python-5th/9781449355722/)